In [25]:
import numpy as np

**1**. (25 points)

The following iterative sequence is defined for the set of positive integers:

- n → n/2 (n is even)
- n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:

13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1

It can be seen that this sequence (starting at 13 and finishing at 1) contains 10 terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at 1.

Which starting number, under one million, produces the longest chain?

NOTE: Once the chain starts the terms are allowed to go above one million.

In [23]:
num = dict()

def collatz(value):
    
    if value == 1:
        return 1
    if value in num.keys():
        return num[value]
    
    length = 0
    if value % 2:
        length = collatz(3 * value + 1)
    else:
        length = collatz(int(value / 2))

    num[value] = length + 1
    return num[value]

lengths = [collatz(x) for x in range(1, int(1e6) + 1)]
print(lengths.index(max(lengths)))


837798


**2** (25 points)


- Perform the median polish to calculate just the *residuals* for this [example](https://mgimond.github.io/ES218/Week11a.html) in Python. 
- Use the matrix `xs` provided
- Display the final result after 3 iterations to 1 decimal place and check if it agrees with 

![img](https://mgimond.github.io/ES218/img/twoway_09.jpg)

In [146]:
xs = np.array([
    (25.3,32.1,38.8,25.4), 
    (25.3,29,31,21.1),
    (18.2,18.8,19.3,20.3),
    (18.3,24.3,15.7,24),
    (16.3,19,16.8,17.5)
]).T

In [147]:
import pandas as pd
np.set_printoptions(precision = 1)

common_eff = round(np.median(xs), 1)
xs -= common_eff
row_eff = np.zeros(len(xs))
col_eff = np.zeros(len(xs.T))

for i in range(3):
    
    #row effects
    row_med = np.median(xs, axis = 1)
    col_eff_med = np.median(col_eff)
    
    row_eff += row_med
    common_eff += col_eff_med
    xs -= row_med[:, None]
    col_eff -= col_eff_med
    
    # column effects
    col_med = np.median(xs, axis = 0)
    row_eff_med = np.median(row_eff)
    
    col_eff += col_med
    common_eff += row_eff_med
    xs -= col_med
    row_eff -= row_eff_med

df = np.array([col_eff] + list(xs))
new_col = np.array([common_eff] + list(row_eff))
df = np.array([new_col] + list(df.T)).T
print(df)


[[20.6  7.6  6.  -0.9  0.2 -3.5]
 [-1.5 -1.4  0.2  0.  -1.   0.7]
 [ 2.5  1.4 -0.2 -3.4  1.  -0.7]
 [-0.3 11.   4.7 -0.  -4.7  0. ]
 [ 0.3 -3.1 -5.9  0.3  2.9  0. ]]


**3**. (50 points)

A Caesar cipher is a very simple method of encoding and decoding data. The cipher simply replaces characters with the character offset by $k$ places. For example, if the offset is 3, we replace `a` with `d`, `b` with `e` etc. The cipher wraps around so we replace `y` with `b`, `z` with `c` and so on. Punctuation, spaces and numbers are left unchanged.

- Write a function `encode` that takes as arguments a string and an integer offset and returns the encoded cipher.
- Write a function `decode` that takes as arguments a cipher and an integer offset and returns the decoded string. 
- Write a function `auto_decode` that takes as argument a cipher and uses a statistical method to guess the optimal offset to decode the cipher, assuming the original string is in English which has the following letter frequency:

```python
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}
```

- Encode the following nursery rhyme using a random offset from 10 to 20, then recover the original using `auto_decode`:

```text
Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane.
```

In [ ]:
def encode(string, offset):
    cipher = ''
    for letter in string:
        asci = ord(letter)
        if asci >= 65 and asci <= 90:
            cipher += chr(65 + (asci - 65 + offset) % 26)
        elif asci >= 97 and asci <= 122:
            cipher += chr(97 + (asci - 97 + offset) % 26)
        else:
            cipher += letter
    return cipher

def decode(cipher, offset):
    string = ''
    for letter in cipher:
        asci = ord(letter)
        if asci >= 65 and asci <= 90:
            string += chr(65 + (asci - 65 - offset) % 26)
        elif asci >= 97 and asci <= 122:
            string += chr(97 + (asci - 97 - offset) % 26)
        else:
            string += letter
    return string

def auto_decode(cipher):
    import math
    freq = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253, 'e': 0.12702, 'f': 0.02228,
    'g': 0.02015, 'h': 0.06094, 'i': 0.06966, 'j': 0.00153, 'k': 0.00772, 'l': 0.04025,
    'm': 0.02406, 'n': 0.06749, 'o': 0.07507, 'p': 0.01929, 'q': 0.00095, 'r': 0.05987,
    's': 0.06327, 't': 0.09056, 'u': 0.02758, 'v': 0.00978, 'w': 0.0236,  'x': 0.0015,
    'y': 0.01974, 'z': 0.00074 
    }   

    # count the letter frequency in the cipher
    freq_cipher = [0] * 26
    for letter in cipher.lower():
        if letter >= 'a' and letter <= 'z':
            freq_cipher[ord(letter) - 97] += 1

    # calculate the cross entropy for each shift of the cipher
    entropy_list = list()
    for offset in range(26):
        entropy = 0.0
        for letter, prob in freq.items():
            entropy += -prob * math.log(prob, 2) * freq_cipher[(ord(letter) - 97 + offset) % 26]
        entropy_list.append(entropy)
    
    return entropy_list.index(max(entropy_list))


